In [6]:
import boto3, re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.model import TensorFlowModel
import tarfile
import keras
import numpy as np
import os
from itertools import groupby
from operator import itemgetter
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn.config import Config
from keras.models import model_from_json
role = get_execution_role()

In [ ]:
# with tarfile.open('/home/ec2-user/SageMaker/keras_model/exported_model/model0100.tar.gz', mode='w:gz') as archive:
#     archive.add("/home/ec2-user/SageMaker/keras_model/savedmodel_0100/", recursive=True)

try:
    !mkdir /tmp/export
    !mkdir /tmp/export/servo
    !mkdir /tmp/export/servo/0
except:
    pass

!cp /home/ec2-user/SageMaker/keras_model/savedmodel_0100/saved_model.pb /tmp/export/servo/0
!cp -r /home/ec2-user/SageMaker/keras_model/savedmodel_0100/variables /tmp/export/servo/variables
with tarfile.open('/tmp/model0100.tar.gz', mode='w:gz') as archive:
     archive.add("/tmp/export",arcname="/export", recursive=True)


In [2]:
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='/tmp/model0100.tar.gz',
                                       key_prefix='model')

sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model0100.tar.gz',
                                  role = role,
                                  framework_version='1.13', py_version='py3',
                                  entry_point = 'train.py', image = '763104351884.dkr.ecr.eu-west-1.amazonaws.com/tensorflow-inference:1.13-cpu' )

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

In [3]:
endpoint_name = "tensorflow-inference-2019-07-09-11-32-10-037"
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

# from tifffile import imsave,imread
# img = imread("190128_LD2_cleaned.tif")

# predictor.predict(dataset)

In [7]:
data_dir = "/home/ec2-user/SageMaker/images/"
class cellConfig(Config):
    NAME = "cell"

    # Adjust to GPU memory
    IMAGES_PER_GPU = 1

    NUM_CLASSES = 2  # Background + cell

    STEPS_PER_EPOCH = 45
    VALIDATION_STEPS = 5

    # Don't exclude based on confidence. Since we have two classes
    # then 0.5 is the minimum anyway as it picks between nucleus and BG
    DETECTION_MIN_CONFIDENCE = 0.5
    #DETECTION_MIN_CONFIDENCE = 0.5

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    #BACKBONE = "resnet50"
    BACKBONE = "resnet101"

    # Input image resizing
    # Random crops of size 512x512
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_SCALE = 2.0

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    #RPN_NMS_THRESHOLD = 0.9
    RPN_NMS_THRESHOLD=0.99

    # How many anchors per image to use for RPN training
    #RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    RPN_TRAIN_ANCHORS_PER_IMAGE = 128

    # Image mean (RGB)
    #MEAN_PIXEL = np.array([43.53, 39.56, 48.22])
    MEAN_PIXEL = np.array([126,126,126])
    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    #MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    MINI_MASK_SHAPE = (100,100)

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    #TRAIN_ROIS_PER_IMAGE = 128
    TRAIN_ROIS_PER_IMAGE = 256

    # Maximum number of ground truth instances to use in one image
    #MAX_GT_INSTANCES = 200
    MAX_GT_INSTANCES = 500
    # Max number of final detections per image
    #DETECTION_MAX_INSTANCES = 400
    DETECTION_MAX_INSTANCES = 1000

class ImageDataset(utils.Dataset):
    def load_images(self, dataset_dir):
        """
        Loads dataset images.
        :param dataset_dir: string, path to dataset directory.
        :return: None
        """
        self.add_class("cell", 1, "cell")

        image_ids = [fn for fn in os.listdir(dataset_dir)
                     if any(fn.endswith(ext) for ext in ['tif', "png"])]

        for image_id in image_ids:
            self.add_image(
                'cell',
                image_id=os.path.splitext(image_id)[0],
                path=os.path.join(dataset_dir, image_id)
            )
            
chunksize = 1
class CellInferenceConfig(cellConfig):
        # Set batch size to 1 to run one image at a time
        GPU_COUNT = 1
        IMAGES_PER_GPU = chunksize
        # Don't resize imager for inferencing
        IMAGE_RESIZE_MODE = "pad64"
        # Non-max suppression threshold to filter RPN proposals.
        # You can increase this during training to generate more propsals.
        RPN_NMS_THRESHOLD = 0.7
        # define the folder path to data for prediction
        global data_dir
        all_files = []
        sub_directory = []
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                relativePath = os.path.relpath(root, data_dir)
                if relativePath == ".":
                    relativePath = ""
                all_files.append(
                    (relativePath.count(os.path.sep), relativePath, file))
        all_files.sort(reverse=True)
        for (count, folder), files in groupby(all_files, itemgetter(0, 1)):
            sub_directory.append(folder)
# dataset = ImageDataset()
# dataset.load_images("/home/ec2-user/SageMaker/images/")
# dataset.prepare()

In [9]:
predict_instance_json = "/home/ec2-user/SageMaker/input_jsons/inputs.json"
config=CellInferenceConfig()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=data_dir)

class ImageDataset(utils.Dataset):
    def load_images(self, dataset_dir):
        """
        Loads dataset images.
        :param dataset_dir: string, path to dataset directory.
        :return: None
        """
        self.add_class("cell", 1, "cell")

        image_ids = [fn for fn in os.listdir(dataset_dir)
                     if any(fn.endswith(ext) for ext in ['tif', "png"])]

        for image_id in image_ids:
            self.add_image(
                'cell',
                image_id=os.path.splitext(image_id)[0],
                path=os.path.join(dataset_dir, image_id)
            )
dataset = ImageDataset()
dataset.load_images(data_dir)
dataset.prepare()
with open(predict_instance_json, "w+") as fp:
    for image_id in dataset.image_ids:
        image = dataset.load_image(image_id)
        active_class_ids = np.zeros([dataset.num_classes], dtype=np.int32)
        source_class_ids = dataset.source_class_ids[dataset.image_info[image_id]["source"]]
        active_class_ids[source_class_ids] = 1
        img, window, scale, padding, crop = utils.resize_image(
                image,
                min_dim=config.IMAGE_MIN_DIM,
                min_scale=config.IMAGE_MIN_SCALE,
                max_dim=config.IMAGE_MAX_DIM,
                mode=config.IMAGE_RESIZE_MODE)
        meta = np.array(
            [image_id] +                  # size=1
            list(image.shape) +           # size=3
            list(img.shape) +             # size=3
            list(window) +                # size=4 (y1, x1, y2, x2) in image cooredinates
            [scale] +                     # size=1
            list(active_class_ids)        # size=num_classes
            )
        anchors = model.get_anchors(image.shape)
        json_data = {'input_image':image.tolist(),
                     'input_image_meta':meta.tolist(),
                     'input_anchors':anchors.tolist()} #,'key':int(image_id)
#         jline = json.dumps(json_data) + "\n"
#         fp.write(jline)

In [11]:
predictor.predict(json_data)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{ "error": "Attempting to use uninitialized value rpn_class_raw/bias\n\t [[{{node rpn_class_raw/bias/read}}]]" }". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2019-07-09-11-32-10-037 in account 102554356212 for more information.